# 기상청 지상 관측자료 - 일자료(기간 조회)
+ API: 기상청 API 허브 https://apihub.kma.go.kr/
+ 참고자료 : https://leedakyeong.tistory.com/entry/Python-Crawling-API-%ED%99%9C%EC%9A%A9%ED%95%98%EC%97%AC-%EA%B3%BC%EA%B1%B0-%EA%B8%B0%EC%83%81-%EA%B4%80%EC%B8%A1-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%88%EB%9F%AC%EC%98%A4%EA%B8%B0
+ 작성: 김지훈
+ 수정: 임경호

### 1. 데이터를 가져올 날짜 설정

In [1]:
import requests
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

# user-defined modules
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

In [2]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date_id FROM weather ORDER BY date_id DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

In [3]:
# 2023년 1월 1일부터 2023년 6월 30일까지 데이터 일괄 저장
start_date = myutil.get_next_day(last_date)
end_date = myutil.get_previous_day()
print(f'날씨 테이블에는 {last_date}까지 데이터가 저장되어 있으며, 이 날짜 이후인 {start_date}부터 {end_date}까지 날씨 데이터를 가져와야 합니다.')

날씨 테이블에는 20230726까지 데이터가 저장되어 있으며, 이 날짜 이후인 20230727부터 20230903까지 날씨 데이터를 가져와야 합니다.


### 2. API를 활용한 데이터 수집 

In [5]:
# url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php?tm1=20151211&tm2=20151214&stn=108&help=1&authKey=853B8q9uTi6dwfKvbq4uqQ'
url = 'https://apihub.kma.go.kr/api/typ01/url/kma_sfcdd3.php'
key = "853B8q9uTi6dwfKvbq4uqQ"      #개인 인증키
region = 143                        # STN(ID) 대구 지역코드
# 컬럼 설정: 해당 API 특성 상 사용할 컬럼명을 미리 지정해줘야 사용가능
col_name =["TM","STN","WS_AVG","WR_DAY","WD_MAX","WS_MAX",
           "WS_MAX_TM", "WD_INS", "WS_INS","WS_INS_TM","TA_AVG","TA_MAX",
           "TA_MAX_TM","TA_MIN","TA_MIN_TM", "TD_AVG", "TS_AVG", "TG_MIN",
           "HM_AVG","HM_MIN","HM_MIN_TM", "PV_AVG", "EV_S", "EV_L", 
           "FG_DUR","PA_AVG","PS_AVG", "PS_MAX", "PS_MAX_TM", "PS_MIN",
           "PS_MIN_TM","CA_TOT","SS_DAY", "SS_DUR", "SS_CMB", "SI_DAY",
           "SI_60M_MAX","SI_60M_MAX_TM","RN_DAY", "RN_D99", "RN_DUR", "RN_60M_MAX",
           "RN_60M_MAX_TM","RN_10M_MAX","RN_10M_MAX_TM", "RN_POW_MAX", "RN_POW_MAX_TM", "SD_NEW",
           "SD_NEW_TM","SD_MAX","SD_MAX_TM", "TE_05", "TE_10", "TE_15",
           "TE_30", "TE_50"
           ]
#데이터 수집 시작 날짜와 끝 날짜를 작성 
# start_date = 20230701 #시작 날짜(수정) 
# end_date = 20230702 # 끝 날짜 (수정)

In [6]:
params ={'tm1' : start_date,     # 시작 날짜
         'tm2' : end_date,       # 끝 날짜
         'stn' : region,    # 지역번호
         'help' : 0,        # 0: 설명이 안나옴, 1: 설명이 나옴 
         'authKey' : key}

response = requests.get(url, params=params, verify=False)

In [7]:
if response.status_code == 200:     # URL GET '200 정상'
    text = response.text
    text = text.split("\n")[5:-2] #앞의 필요없는  5줄 부분과 뒤의 필요없는 2줄 부분을 제거
    
    df_weather = pd.DataFrame(text)[0].str.split(expand=True) #데이터프레임 컬럼으로 변경
    df_weather.columns = col_name

In [8]:
df_weather

,TM,STN,WS_AVG,WR_DAY,WD_MAX,WS_MAX,WS_MAX_TM,WD_INS,WS_INS,WS_INS_TM,...,RN_POW_MAX_TM,SD_NEW,SD_NEW_TM,SD_MAX,SD_MAX_TM,TE_05,TE_10,TE_15,TE_30,TE_50
0,20230727,143,1.3,1102,29,2.9,1256,29,4.8,1311,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
1,20230728,143,1.1,964,27,3.3,1335,27,4.6,1327,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
2,20230729,143,1.1,943,29,2.8,1453,32,5.4,1446,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
3,20230730,143,1.5,1298,14,3.3,1601,20,5.5,1744,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
4,20230731,143,1.5,1298,9,4.0,1830,32,7.8,1425,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
5,20230801,143,1.9,1640,11,3.4,1231,11,5.3,1150,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
6,20230802,143,1.5,1314,16,3.1,1813,16,4.9,1807,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
7,20230803,143,1.8,1537,27,4.3,1408,25,8.0,1402,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
8,20230804,143,1.1,917,5,3.0,1535,34,4.2,1234,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0
9,20230805,143,1.5,1258,32,3.7,1645,29,6.1,1638,...,-9,-9.0,-9,-9.0,-9,-99.0,-99.0,-99.0,-99.0,-99.0


### 3. 데이터 전처리

In [9]:
#데이터 전처리 
df_weather= df_weather[['TM','TA_AVG','TA_MAX','TA_MIN']] #필요한 부분만 추출>|날짜|평균기온|최고기온|최저기온|
df_weather.columns = ['date_id','ta_avg','ta_max','ta_min'] #컬럼명 변경 > SQL과 똑같이 설정

#타입 변경
df_weather['ta_avg'] = df_weather['ta_avg'].astype(float) #실수형으로 변경
df_weather['ta_max'] = df_weather['ta_max'].astype(float) #실수형으로 변경
df_weather['ta_min'] = df_weather['ta_min'].astype(float) #실수형으로 변경

#기온 편차 컬럼 생성
df_weather['ta_diff'] = df_weather['ta_max']-df_weather['ta_min'] 
df_weather

,date_id,ta_avg,ta_max,ta_min,ta_diff
0,20230727,27.9,32.9,22.8,10.1
1,20230728,29.4,34.5,24.6,9.9
2,20230729,30.1,35.3,24.8,10.5
3,20230730,29.8,35.5,25.0,10.5
4,20230731,29.6,34.4,24.7,9.7
5,20230801,29.7,34.8,24.9,9.9
6,20230802,30.0,34.7,23.9,10.8
7,20230803,31.5,37.7,27.0,10.7
8,20230804,30.5,34.7,26.0,8.7
9,20230805,31.5,37.2,25.7,11.5


### 4. 테이블에 데이터 저장 (mdw.weather)

In [10]:
import datetime as dt
dt_now = dt.datetime.now()
dt_now_str = dt_now.strftime('%Y-%m-%d %H:%M:%S')

user_name = 'lkh'

In [11]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_weather.itertuples():
    sql = "insert into weather (date_id, ta_avg, ta_max, ta_min, ta_diff, create_datetime, create_id) \
                        values (%s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], dt_now_str, user_name))
conn.commit()
conn.close()